# Fashion‑MNIST: Unsupervised Pipeline (PCA/SVD + Clustering)

This notebook builds a full unsupervised pipeline on the **Fashion‑MNIST** dataset:

1) Data understanding & preprocessing  
2) Dimensionality reduction with **PCA** and **TruncatedSVD**  
3) Clustering with **K‑Means** and **DBSCAN**  
4) Analysis & interpretation (plots + short insights)

**Goal:** show how structure appears in image data without labels, then use labels only to interpret quality.


### Quick setup
Install the basics if needed:

```bash
pip install numpy pandas plotly scikit-learn pillow
# optional: one of these loaders (either works)
pip install tensorflow            # Keras loader
# OR DO this instead
pip install openml                # fetch_openml backend
```


In [ ]:

# Imports
import numpy as np, pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import fetch_openml

import warnings, sys, math, os, random
np.random.seed(42)
random.seed(42)

# Class names for readability
LABELS = {0:'T-shirt/top',1:'Trouser',2:'Pullover',3:'Dress',4:'Coat',5:'Sandal',6:'Shirt',7:'Sneaker',8:'Bag',9:'Ankle boot'}

# Helpers for plots
def heatmap_grid(imgs, titles=None, rows=2, cols=5, main_title=""):
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=titles)
    r = c = 1
    for i, img in enumerate(imgs):
        vmax = 255 if img.max() > 1 else 1
        fig.add_trace(go.Heatmap(z=img, colorscale='gray', zmin=0, zmax=vmax, showscale=False), row=r, col=c)
        c += 1
        if c == cols+1: r, c = r+1, 1
    fig.update_layout(height=520, title_text=main_title)
    fig.update_xaxes(showticklabels=False); fig.update_yaxes(showticklabels=False)
    fig.show()


## Phase 1 — Data understanding & preprocessing
**Why:** sanity‑check the dataset, normalize to `[0,1]`, and flatten to vectors for PCA/cluster.


In [ ]:

# Try Keras loader first  if unavailable, fall back to OpenML
X_train = y_train = X_test = y_test = None
try:
    from tensorflow.keras.datasets import fashion_mnist
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    print('Loaded with Keras:', X_train.shape, X_test.shape)
except Exception as e:
    print('Keras loader not available or failed:', e)
    try:
        ds = fetch_openml(data_id=40996, as_frame=False)  # Fashion-MNIST
    except Exception:
        ds = fetch_openml(name='Fashion-MNIST', version=1, as_frame=False)
    X = ds['data'].astype('float32')/255.0    # (70000, 784) in [0,1]
    y = ds['target'].astype(int)
    X_train, X_test = X[:60000].reshape(-1,28,28), X[60000:].reshape(-1,28,28)
    y_train, y_test = y[:60000], y[60000:]
    print('Loaded with OpenML:', X_train.shape, X_test.shape)

# Normalize and flatten
X_train_f = X_train.astype('float32')/255.0 if X_train.max()>1 else X_train.astype('float32')
X_test_f  = X_test.astype('float32')/255.0 if X_test.max()>1 else X_test.astype('float32')
X_train_flat = X_train_f.reshape(len(X_train_f), -1)
X_test_flat  = X_test_f.reshape(len(X_test_f), -1)
print('Flattened:', X_train_flat.shape, X_test_flat.shape)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Loaded with Keras: (60000, 28, 28) (10000, 28, 28)
Flattened: (60000, 784) (10000, 784)


In [ ]:

# Class balance
bal = (pd.Series(y_train).map(LABELS).value_counts()
       .rename_axis('class').reset_index(name='count').sort_values('class'))
px.bar(bal, x='class', y='count', title='Train class counts').show()

# One sample per class
idxs = [np.where(y_train==c)[0][0] for c in range(10)]
imgs = [X_train[i] for i in idxs]
heatmap_grid(imgs, [LABELS[c] for c in range(10)], main_title='Fashion-MNIST: one sample per class')


In [ ]:

# Pixel intensity distribution sample
flat = X_train_f.reshape(-1)
sample = flat[np.random.choice(flat.size, 120_000, replace=False)]
px.histogram(x=sample, nbins=50, labels={'x':'intensity','y':'count'},
             title='Pixel intensity distribution (0–1)').show()

# Per-class mean brightness
means = (pd.DataFrame({'label': y_train, 'mean_int': X_train_flat.mean(axis=1)})
         .replace({'label': LABELS}).groupby('label', as_index=False)['mean_int'].mean()
         .sort_values('mean_int'))
px.bar(means, x='label', y='mean_int', labels={'label':'','mean_int':'mean intensity'},
       title='Average image brightness per class').show()

# Overall & class average images
overall_avg = X_train_f.mean(axis=0)
px.imshow(overall_avg, color_continuous_scale='gray', title='Overall average image')\
  .update_xaxes(showticklabels=False).update_yaxes(showticklabels=False).show()

avg_imgs = [X_train_f[y_train==c].mean(axis=0) for c in range(10)]
heatmap_grid(avg_imgs, [LABELS[c] for c in range(10)],
             main_title='Class‑average images (prototypes)')


## Phase 2 — Dimensionality reduction (PCA & SVD)
**Why:** compress images while keeping most variance; reveals structure and speeds up clustering.


In [ ]:

# PCA fit and explained variance
pca_k = 200
pca = PCA(n_components=pca_k, svd_solver='randomized', random_state=42)
pca.fit(X_train_flat)
evr = pca.explained_variance_ratio_
cum = np.cumsum(evr)
k95 = int(np.argmax(cum >= 0.95)) + 1
fig = go.Figure()
fig.add_trace(go.Scatter(y=evr, mode='lines', name='per-component EVR'))
fig.add_trace(go.Scatter(y=cum, mode='lines', name='cumulative EVR', yaxis='y2'))
fig.update_layout(title=f'PCA explained variance (k={pca_k})',
                  xaxis_title='component index',
                  yaxis=dict(title='EVR'),
                  yaxis2=dict(title='Cumulative EVR', overlaying='y', side='right', range=[0,1]))
fig.add_vline(x=k95, line_dash='dash'); fig.add_annotation(x=k95, y=0.5, text=f'95% @ {k95}', showarrow=True)
fig.show()
print('k for ~95% variance:', k95)


k for ~95% variance: 189


In [ ]:

# 2D/3D PCA scatter subsample
np.random.seed(42)
sub_ix = np.random.choice(len(X_train_flat), 10_000, replace=False)
Z = pca.transform(X_train_flat[sub_ix])
df2 = pd.DataFrame({'pc1':Z[:,0], 'pc2':Z[:,1], 'label':[LABELS[int(i)] for i in y_train[sub_ix]]})
px.scatter(df2, x='pc1', y='pc2', color='label', opacity=0.6, title='PCA 2D (10k samples)').show()

df3 = pd.DataFrame({'pc1':Z[:,0], 'pc2':Z[:,1], 'pc3':Z[:,2], 'label':[LABELS[int(i)] for i in y_train[sub_ix]]})
px.scatter_3d(df3, x='pc1', y='pc2', z='pc3', color='label', opacity=0.5, title='PCA 3D (10k samples)').show()


In [ ]:

# Reconstructions at different k
test_ix = [np.where(y_test==c)[0][0] for c in range(10)]
orig = X_test_f[test_ix]; orig_flat = orig.reshape(len(orig), -1)
heatmap_grid([o for o in orig], [LABELS[c] for c in range(10)], main_title='Original (one per class)')
for k in [10, 50, 100, 200]:
    p = PCA(n_components=k, svd_solver='randomized', random_state=42).fit(X_train_flat)
    rec = p.inverse_transform(p.transform(orig_flat)).reshape(-1,28,28)
    heatmap_grid([img for img in rec], [f'{LABELS[c]} (k={k})' for c in range(10)],
                 main_title=f'PCA reconstructions @ k={k}')


In [ ]:

# Reconstruction MSE vs k (PCA) on a subset
np.random.seed(0)
ix2 = np.random.choice(len(X_test_flat), 2000, replace=False)
Xsub = X_test_flat[ix2]
def mse(a,b): return float(np.mean((a-b)**2))
k_grid = [5,10,20,30,50,75,100,150,200]
mse_pca = []
for k in k_grid:
    p = PCA(n_components=k, svd_solver='randomized', random_state=42).fit(X_train_flat)
    rec = p.inverse_transform(p.transform(Xsub))
    mse_pca.append(mse(Xsub, rec))
px.line(x=k_grid, y=mse_pca, markers=True, labels={'x':'# components (k)','y':'MSE'},
        title='Reconstruction error vs k (PCA)').show()


In [ ]:

# TruncatedSVD compare
svd = TruncatedSVD(n_components=200, random_state=42).fit(X_train_flat)
evr_svd = svd.explained_variance_ratio_
fig = go.Figure()
fig.add_trace(go.Scatter(y=evr, name='PCA EVR', mode='lines'))
fig.add_trace(go.Scatter(y=evr_svd, name='TruncatedSVD EVR', mode='lines'))
fig.update_layout(title='Per-component explained variance: PCA vs TruncatedSVD',
                  xaxis_title='component', yaxis_title='EVR')
fig.show()

mse_svd = []
for k in k_grid:
    m = TruncatedSVD(n_components=k, random_state=42).fit(X_train_flat)
    rec = m.inverse_transform(m.transform(Xsub))
    mse_svd.append(mse(Xsub, rec))
fig = go.Figure()
fig.add_trace(go.Scatter(x=k_grid, y=mse_pca, name='PCA', mode='lines+markers'))
fig.add_trace(go.Scatter(x=k_grid, y=mse_svd, name='TruncatedSVD', mode='lines+markers'))
fig.update_layout(title='Reconstruction MSE vs k: PCA vs TruncatedSVD',
                  xaxis_title='# components (k)', yaxis_title='MSE')
fig.show()


## Phase 3 — Clustering (K‑Means & DBSCAN)
**Why:** group images without labels in PCA space, then interpret clusters with labels. We use PCA(100, whiten) for speed and better distance scaling.


In [ ]:

# PCA for clustering (100 comps, whiten=True)
pca_clust = PCA(n_components=100, svd_solver='randomized', whiten=True, random_state=42)
Z_train = pca_clust.fit_transform(X_train_flat)
Z_test  = pca_clust.transform(X_test_flat)
print('Z shapes:', Z_train.shape, Z_test.shape)


Z shapes: (60000, 100) (10000, 100)


In [ ]:

# K-Means sweep on subset: inertia + silhouette
np.random.seed(42)
ix = np.random.choice(len(Z_train), 20000, replace=False)
Zt, yt = Z_train[ix], y_train[ix]
K_range = list(range(6,13))
inertia, sil = [], []
for K in K_range:
    km = KMeans(n_clusters=K, init='k-means++', n_init=10, random_state=42)
    lab = km.fit_predict(Zt)
    inertia.append(km.inertia_)
    sil.append(silhouette_score(Zt, lab))

fig = go.Figure()
fig.add_trace(go.Scatter(x=K_range, y=inertia, mode='lines+markers', name='inertia (≤ better)'))
fig.add_trace(go.Scatter(x=K_range, y=sil, mode='lines+markers', name='silhouette (≥ better)', yaxis='y2'))
fig.update_layout(title='K-Means sweep on PCA(100)',
                  xaxis_title='K',
                  yaxis=dict(title='inertia'),
                  yaxis2=dict(title='silhouette', overlaying='y', side='right', range=[0,1]))
fig.show()
bestK = K_range[int(np.argmax(sil))]
print('Best K by silhouette on subset =', bestK)


Best K by silhouette on subset = 12


In [ ]:

# Final K-Means fit + evaluation vs labels (for interpretation only)
km_final = KMeans(n_clusters=bestK, init='k-means++', n_init=20, random_state=42)
train_clusters = km_final.fit_predict(Z_train)
test_clusters = km_final.predict(Z_test)

# map cluster -> majority label (train), then pseudo-accuracy on test
def majority_map(clusters, true_y):
    df = pd.DataFrame({'c':clusters, 'y':true_y})
    return df.groupby('c')['y'].agg(lambda s: s.value_counts().idxmax()).to_dict()
c2y = majority_map(train_clusters, y_train)
test_pred_labels = np.vectorize(c2y.get)(test_clusters)

ari = adjusted_rand_score(y_test, test_clusters)
nmi = normalized_mutual_info_score(y_test, test_clusters)
acc_like = (test_pred_labels == y_test).mean()
print(f'ARI={ari:.3f}  NMI={nmi:.3f}  Purity-like accuracy={acc_like:.3f}')

# confusion-style heatmaps (true x cluster)
ct = pd.crosstab(pd.Series(y_test, name='true'), pd.Series(test_clusters, name='cluster'))
ct = ct.reindex(sorted(ct.columns), axis=1)
ct.index = [LABELS[i] for i in ct.index]
px.imshow(ct, text_auto=True, aspect='auto', title='K-Means: true labels × cluster (counts)').show()
ct_norm = ct.div(ct.sum(axis=1), axis=0)
px.imshow(ct_norm.values, color_continuous_scale='Blues', zmin=0, zmax=1, aspect='auto',
          title='K-Means: row-normalized (fraction per true class)').show()

# visualize clusters in 2D (first two PCs of Z_train for 10k points)
df_vis = pd.DataFrame({'pc1':Z_train[:10000,0], 'pc2':Z_train[:10000,1], 'cluster':train_clusters[:10000].astype(str)})
px.scatter(df_vis, x='pc1', y='pc2', color='cluster', opacity=0.6, title='K-Means clusters on PCA(100) — first 10k').show()

# purity per cluster & best recall per class
purity_cluster = (ct.max(axis=0) / ct.sum(axis=0)).rename('purity').reset_index()
px.bar(purity_cluster, x='cluster', y='purity', title='Cluster purity (share of top true class)').show()
best_recall_class = (ct.max(axis=1) / ct.sum(axis=1)).rename('best_cluster_recall').reset_index()
best_recall_class['true'] = [LABELS[i] for i in range(10)]
px.bar(best_recall_class, x='true', y='best_cluster_recall', title='Per-class best cluster recall').show()

# cluster centroids as images (approximate prototypes)
centers_rec = pca_clust.inverse_transform(km_final.cluster_centers_).reshape(bestK,28,28)
titles = [f'C{k}' for k in range(bestK)]
heatmap_grid(list(centers_rec), titles, rows=math.ceil(bestK/5), cols=5, main_title='Cluster centroids (reconstructed)')


ARI=0.308  NMI=0.502  Purity-like accuracy=0.557


In [ ]:

# DBSCAN on a subset (heavier). We'll estimate eps via k-distance curve.
np.random.seed(0)
ix_db = np.random.choice(len(Z_train), 10000, replace=False)
Zd, yd = Z_train[ix_db], y_train[ix_db]
min_pts = 5
nbrs = NearestNeighbors(n_neighbors=min_pts).fit(Zd)
dists, _ = nbrs.kneighbors(Zd)
k_d = np.sort(dists[:, -1])
px.line(y=k_d, title='k-distance curve (k=5) — look for the knee').show()

# try a few eps candidates
cands = list(np.percentile(k_d, [85, 90, 92, 94, 96, 98]))
rows = []
for eps in cands:
    db = DBSCAN(eps=float(eps), min_samples=min_pts, n_jobs=-1)
    lab = db.fit_predict(Zd)
    K = len(set(lab)) - (1 if -1 in lab else 0)
    if K >= 2 and len(np.unique(lab[lab!=-1]))>1:
        s = silhouette_score(Zd[lab!=-1], lab[lab!=-1])
    else:
        s = -1
    rows.append({'eps': float(eps), 'clusters': K, 'silhouette': s, 'noise_frac': float((lab==-1).mean())})
res_df = pd.DataFrame(rows)
px.scatter(res_df, x='eps', y='silhouette', size='clusters', hover_data=['noise_frac'], title='DBSCAN sweep (subset)').show()

# pick best by silhouette, visualize
if not res_df.empty and res_df['clusters'].max()>=2:
    eps_best = float(res_df.sort_values('silhouette', ascending=False).iloc[0]['eps'])
    db_best = DBSCAN(eps=eps_best, min_samples=min_pts, n_jobs=-1)
    lab_best = db_best.fit_predict(Zd)
    lab_plot = np.where(lab_best==-1, 'noise', lab_best.astype(str))
    dfv = pd.DataFrame({'pc1':Zd[:,0], 'pc2':Zd[:,1], 'cluster':lab_plot})
    px.scatter(dfv, x='pc1', y='pc2', color='cluster', opacity=0.6,
               title=f'DBSCAN clusters (subset) — eps={eps_best:.2f}, minPts={min_pts}').show()


## Phase 4 — Analysis & Interpretation
- Most pixel mass is near 0 → edges/shapes carry variance.  
- ~95% variance around 180–190 PCs; **100 PCs** works well for clustering speed/quality.  
- K‑Means finds meaningful groups (e.g., **Trousers**, **Ankle boots**). **Shirt vs T‑shirt** overlap more.  
- DBSCAN is sensitive here; it tends to merge many points or mark noise unless `eps` is tuned.


In [ ]:
\